In [111]:
import sys
import time
import numpy as np
import random
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import heapq

In [168]:
 """ Clase que implementa el individuo y sus operadores. El cromosoma de un individuo es una lista de caracteres,
       cada elemento de la lista es un gen cuyos alelos (caracteres) posibles se indican en allele_pool"""

class Individual:   
    
    def __init__(self, chromosome, allele_pool):  # el constructor recibe el cromosoma  y el pool de alelos posibles
        self.chromosome = chromosome[:]
        self.allele_pool = allele_pool
        self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
        
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:], allele_pool)
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:], allele_pool)
        return [ind1, ind2]   
    
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1, allele_pool)
        ind2 = Individual(chromosome2, allele_pool)
        return [ind1, ind2] 

    def mutate_position(self):
        "Cambia aleatoriamente el alelo de un gen."
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = allele_pool[random.randrange(0,len(allele_pool))]
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome, allele_pool)
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome)
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome, allele_pool)

## Poblacion tipo string

In [113]:
def init_population(pop_number, chromosome_size, allele_pool):
    
    num_alleles = len(allele_pool)
    population = []
    for i in range(pop_number):
        new_chromosome = [allele_pool[random.randrange(0, num_alleles)] for j in range(chromosome_size)]
        population.append( Individual(new_chromosome, allele_pool) )
    return population

In [114]:
def display(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([''.join(population[i].chromosome),population[i].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

In [135]:
def display_m(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([''.join(population[i][0].chromosome),population[i][1].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

## Selecciona los padres mediante operadores: Ruleta/ Torneo

### Ruleta

In [115]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

### Torneo

In [116]:
def select_parent_torneo(population,size_torneo):
    
    # Escoje el primer padre
    list_indiv=[]
    x1 = np.random.permutation(len(population) )
    y1= x1[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y1[i]].fitness)
    
    iParent1=np.argmax(list_indiv)
    
    # Escoje el segundo padre, desconsiderando el primer padre   
    x2 = np.delete(x1, iParent1)
    x2 = np.random.permutation(x2)
    list_indiv=[]
    y2= x2[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y2[i]].fitness)
    iParent2=np.argmax(list_indiv)
    
    return (population[x1[iParent1]],population[x2[iParent2]])

## Iniciando una poblacion

In [127]:
target_string = 'Inteligencia Artificial en la PUCP'


# construye el pool de alelos con los caracteres en mayuscula, minuscula y espacio en blanco
allele_pool = []
allele_pool.extend( [chr(x) for x in range(65, 91)] )   # caracteres ASCII en mayuscula
allele_pool.extend( [chr(x) for x in range(97, 123)] )  # caracteres ASCII en minuscula
allele_pool.extend(' ')  # espacio en blanco

# Inicializa una poblacion inicial de forma aleatoria
num_individuals = 7
population = init_population(num_individuals, len(target_string), allele_pool)

#display(population) #Imprime la primera poblacion 

 ## Probando los Operadores

In [128]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=19
display(population)

,Poblacion,fitness
0,NarelKBttk MOFAHDMJdhhTTWQGvzF tvP,2
1,LyJmkDqdQYvW cRARFZIKeeLvyhXdrxglm,23
2,oCrpDCTWsmiNNHbORKXVNsjScjeVQNNeSK,11
3,yZKNiNOikJcXjAHJbSAhuWNUPRNaEvimmL,17
4,xLPdyLvnYYG JsQEBqGDgyM RNxqzrZDcD,22
5,vCOPNpz bRbfu SIjHmdrIiCseOETsMBok,5
6,oeCtoVpfLXPJlKsPFbeJoBQIbwDdPucZyk,19


- Llamando a torneo

In [119]:
size_torneo = 3

winner1,winner2=select_parent_torneo(population,size_torneo)
print(''.join(winner1.chromosome),winner1.fitness)
print(''.join(winner2.chromosome),winner2.fitness)


PXRUbAuiDFbquKYhRywuXLfRZkphoqdHoq 23
hsTbSskhROuJtQBJuiDygGgXzJYrGVjVMd 19


- Llamando a ruleta

In [120]:
winner1,winner2=select_parents_roulette(population)
print(''.join(winner1.chromosome),winner1.fitness)
print(''.join(winner2.chromosome),winner2.fitness)

 mkyqSmMVLNxp spQfIMifpBRbsYWLdfCS 11
PXRUbAuiDFbquKYhRywuXLfRZkphoqdHoq 23


## Probando Cruzamiento

In [175]:
popsize = len(population)
## Selecciona las parejas de padres para cruzamiento 
mating_pool = []
for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 

In [176]:
display_m(mating_pool)

,Poblacion,fitness
0,LyJmkDqdQYvW cRARFZIKeeLvyhXdrxglm,17
1,oeCtoVpfLXPJlKsPFbeJoBQIbwDdPucZyk,2
2,oCrpDCTWsmiNNHbORKXVNsjScjeVQNNeSK,19


In [177]:
# Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
offspring_population = []
for i in range(len(mating_pool)): 
    offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) )
    #offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )

In [178]:
display(offspring_population)

,Poblacion,fitness
0,LyJmkDqdQYvWjAHJbSAhuWNUPRNaEvimmL,-1
1,yZKNiNOikJcX cRARFZIKeeLvyhXdrxglm,-1
2,oeCtoVpfLk MOFAHDMJdhhTTWQGvzF tvP,-1
3,NarelKBttXPJlKsPFbeJoBQIbwDdPucZyk,-1
4,oCrpDCpfLXPJlKsPFbeJoBQIbwDdPucZyk,-1
5,oeCtoVTWsmiNNHbORKXVNsjScjeVQNNeSK,-1


## Probando Mutacion

In [179]:
pmut=0.1
## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
for i in range(len(offspring_population)):
    if random.uniform(0, 1) < pmut: 
       # offspring_population[i] = offspring_population[i].mutate_swap()
       offspring_population[i] = offspring_population[i].mutate_position()

In [180]:
display(offspring_population)

,Poblacion,fitness
0,LyJmkDqdQYvWjAHJbSAhuWNUPRNaEvimmL,-1
1,yZKNiNOikJcX cRARFZIKeeLvyhXdrxglm,-1
2,oeCtoVpfLk MOFAHDMJrhhTTWQGvzF tvP,-1
3,NarelKBttXPJlKsPFbeJoBQIbwDdPucZyk,-1
4,oCrpDCpfLXPJlKsPFbeJoBQIAwDdPucZyk,-1
5,oeCtoVTWsmiNNHbORKXVNsTScjeVQNNeSK,-1
